# Un ejemplo un poco más complicado

Vamos a hacer un caso muy simplificado de una cuenta de un banco.

## Índice
- Protocolo de mensajería
  + De entrada
  + De salida
  + Eventos
- Cuentas y transferencias
  + Actor cuenta
  + Interfaz cuenta
  + Definición de una transferencia
- Implementaciones
  + Logica de negocio: actualización del balance
  + Publicación de eventos
 

###  Se importan la librerias de akka 

In [1]:
import $ivy.`com.typesafe.akka::akka-actor:2.5.14`

import $ivy.$                                     

### Implicitos necesarios

In [2]:
import scala.concurrent.ExecutionContext 
import java.util.concurrent.Executors
import akka.util.Timeout
import scala.concurrent.duration._

implicit val ec  = ExecutionContext.fromExecutorService( Executors.newFixedThreadPool( 20 ) )
implicit val timeout = Timeout( 5 seconds )   

object Types {    
   type Balance = Int
   type Amount  = Int
   type IdAccount = String
}

import scala.concurrent.ExecutionContext 

import java.util.concurrent.Executors

import akka.util.Timeout

import scala.concurrent.duration._


ec: concurrent.ExecutionContextExecutorService = scala.concurrent.impl.ExecutionContextImpl$$anon$1@129b1d
timeout: akka.util.Timeout = Timeout(5 seconds)
defined object Types

---

---
## Se define el protocolo de mensajería

### Protocolo de entrada

In [5]:
import Types._

sealed trait AccountIn

// Commands

sealed trait AccountCommand extends AccountIn { 
    val amount : Balance
}

final case class Withdrawal(amount : Amount) extends AccountCommand 
final case class Income(amount : Amount) extends AccountCommand 


// Queries

sealed trait AccountQuery extends AccountIn

final case object GetBalance extends AccountQuery

import Types._


defined trait AccountIn
defined trait AccountCommand
defined class Withdrawal
defined class Income
defined trait AccountQuery
defined object GetBalance

### Protocolo de salida

In [3]:
import Types._ 

sealed trait AccountOut

final case class CurrentBalance( balance: Balance ) extends AccountOut

import Types._ 


defined trait AccountOut
defined class CurrentBalance

### Eventos

In [6]:
import Types._

sealed trait AccountEvent {    
    val idAccount : IdAccount
    val amount: Amount
}

case class WithdrawalCreated( 
                              val idAccount: IdAccount, 
                              val amount : Amount 
                            ) extends AccountEvent

case class IncomeCreated( 
                          val idAccount: IdAccount, 
                          val amount : Amount 
                        ) extends AccountEvent

import Types._


defined trait AccountEvent
defined class WithdrawalCreated
defined class IncomeCreated

---

---
## Cuentas y transferencias
### Actor 'Cuenta'

In [7]:
import akka.actor._
import scala.collection.mutable.Queue
import scala.util._
import Types._

class ActorAccount( 
                    private val updateBalance : (Amount, Balance) => Try[Balance], 
                    private val queueCQRS: Queue[AccountEvent] 
                  ) extends Actor {
    
    
    val id = self.path.name
    
    var balance : Int = 0
    
    override def receive = {
        
        case command : AccountCommand => manageCommads( command )
        case querry  : AccountQuery   => manageQueries( querry )
        case other                    => unhandled( other )
        
    }
    
    private def manageCommads( command: AccountCommand ) : Unit = {
        
        command match {
            case Withdrawal( amount ) => execUpdateBalance( -1 * amount, command)
            case Income( amount )     => execUpdateBalance( amount, command )
            
        }           
        
    }
    
    private def execUpdateBalance( amount : Amount, command : AccountCommand) = {        
        updateBalance( amount, balance ) match {            
            case Success( newBalance ) => {
                balance = newBalance
                responseBalance( balance )
                sendEvent( command )
                
            }
            case Failure(  error ) => sender() ! Status.Failure( error )         
        }
    }
    
    private def sendEvent( command: AccountCommand ) {
        
        val event : AccountEvent = command match {
            case Withdrawal( amount ) => WithdrawalCreated( id, amount ) 
            case Income( amount )     => IncomeCreated( id, amount ) 
        }
        
        queueCQRS.enqueue( event )
        
    }
    
    private def manageQueries( queries : AccountQuery ) : Unit = queries match {
        case GetBalance => responseBalance( balance )
    }
    
    private def responseBalance( bal : Balance ) = sender() !  CurrentBalance( bal )    
    
}

import akka.actor._

import scala.collection.mutable.Queue

import scala.util._

import Types._


defined class ActorAccount

### Interfaz 'Cuenta'

In [8]:
import scala.concurrent.Future
import Types._

trait Account {
   def makeWithdrawal( amount : Amount ) : Future[Balance] 
   def makeIncome( amount : Amount ) : Future[Balance] 
   def getBalance: Future[Balance]
}

object Account {
    
    import akka.pattern._
    import akka.actor._
    import akka.util.Timeout
    
    private def toBalance( responseActor : Future[Any] )( implicit ec : ExecutionContext ) : Future[Balance] = {
        responseActor.mapTo[CurrentBalance].map( _.balance )
    }
    
    def apply( account :ActorRef )
                    ( implicit ec : ExecutionContext, timeout : Timeout ) = new Account {
      
      def makeWithdrawal( amount : Amount ) : Future[ Balance] =  toBalance {
          account ? Withdrawal( amount ) 
      }

      def makeIncome( amount : Amount ) : Future[Balance] =  toBalance {
          account ? Income( amount ) 
      }
        
      def getBalance : Future[Balance] =  toBalance {
          account ? GetBalance
      }  
        
    }
}



import scala.concurrent.Future

import Types._


defined trait Account
defined object Account

### Definición de una transferencia

Se simula una operación/compensacion siguiendo el patrón sagas

In [9]:
import Types._

object Transfer {
    
    import scala.concurrent._
    
    def transfer( from : Account, to: Account )( amount : Amount )( implicit ec : ExecutionContext) = {
        
        for {
            
             _  <- from.makeWithdrawal( amount ) 
            res <- to.makeIncome( amount )
                    .map( _ => true )
                    .recoverWith{ 
                        case _ => from.makeIncome( amount ).map( _=> false ) 
                    }
        } yield( res )
        
   }
    
}

import Types._


defined object Transfer

---

----
## Implementaciones

### Lógica de negocio
Se define una lógica de negocio simple. En este caso no se admiten descubiertos, pero por ejemplo se pueden implementar diferentes lógicas como un porcentaje de descubierto dependiendo del balance. 
> El objetivo final es que la lógica puede estar separada del actor y puede ser validada y probada aparte

In [10]:
import scala.util._
import Types._

implicit val updateBalance : (Amount,Balance) => Try[Balance] = ( amount, balance ) => {
   
    val newBalance = amount + balance
    
    if( newBalance >= 0 ) {
    
        Success( newBalance )
        
    } else {
        
        Failure( new IllegalStateException( s"It should not be in red( ${newBalance} )" ) )
    }
    
}

import scala.util._

import Types._


updateBalance: (Amount, Balance) => Try[Balance] = $sess.cmd9Wrapper$Helper$$Lambda$3413/1242457962@215a85e1

### Indirección de publicación de eventos
Se define una cola que será la indirección de publicación de eventos.
En este caso para esta prueba será una cola mitable de Scala.   
> En un sistema real puede ser un akka stream con su fuente '_materializada_' en una cola

In [11]:
import scala.collection.mutable.Queue

val queueCQRS = Queue[AccountEvent]()

import scala.collection.mutable.Queue


queueCQRS: Queue[AccountEvent] = Queue()

---

---
## Probandolo todo

### _Testing: Utilidades_

> **Sólo para motivos de testing**. Espera el resultado de un futuro

In [12]:
object TestUtil {
    
    import scala.concurrent._, duration._
    import akka.pattern._
    import akka.util.Timeout


    val tm = 5 seconds
    implicit val timeout = Timeout( tm )

    def result[T]( future : => Future[T] ) = Try {
        Await.result( future, tm )
    }
    
}

defined object TestUtil

---
### Iniciando el entorno

#### Se crea el sistema de actores

In [13]:
val system = akka.actor.ActorSystem.create( "test-1" )

system: ActorSystem = akka://test-1

#### Se crean dos actores cuenta

In [14]:
import akka.actor.Props

val accountOneActor = system.actorOf (
        Props( new ActorAccount(  updateBalance, queueCQRS )),  
       "accountOne"  
)

val accountTwoActor = system.actorOf (
        Props( new ActorAccount( updateBalance, queueCQRS ) ),
        "accountTwo"
)

import akka.actor.Props


accountOneActor: ActorRef = Actor[akka://test-1/user/accountOne#374386396]
accountTwoActor: ActorRef = Actor[akka://test-1/user/accountTwo#-1598936892]

#### Se crean dos 'entidades' cuenta
A partir de los dos actores cuenta se crean las instancias del interfaz "_Cuenta_"

In [15]:
val accountOne = Account( accountOneActor )
val accountTwo = Account( accountTwoActor )

accountOne: AnyRef with Account = $sess.cmd7Wrapper$Helper$Account$$anon$1@5dfe9e1f
accountTwo: AnyRef with Account = $sess.cmd7Wrapper$Helper$Account$$anon$1@3b0cdd32

#### Se hace un ingreso incial a las dos cuentas

Se obtiene el resutlado de los dos balances y se calcula el total del dinero (la suma de los dos balances)

> Aquí se hace `Await` sólo por motivos de testing

In [16]:

TestUtil.result{
    
    accountOne.makeIncome( 1000 ).zipWith( accountTwo.makeIncome( 1000 ) ){
        ( b1, b2 ) => (b1, b2, b1 +b2) 
    }
    
}

res15: Try[(Balance, Balance, Int)] = Success((1000, 1000, 2000))

#### Se comprueban los eventos

In [17]:
queueCQRS.toList ; queueCQRS.clear

res16_0: List[AccountEvent] = List(IncomeCreated(accountOne,1000), IncomeCreated(accountTwo,1000))

---
### _Probando, probando_

#### Funciones de utilidades
Para poder testear transferencias de una cuenta a otra de una manera más cómoda

In [18]:
val transfersOneToTwo =  Transfer.transfer( accountOne, accountTwo)( _ )
val transfersTwoToOne =  Transfer.transfer( accountTwo, accountOne)( _ )

transfersOneToTwo: Amount => Future[Boolean] = $sess.cmd17Wrapper$Helper$$Lambda$3657/1314747481@116f917d
transfersTwoToOne: Amount => Future[Boolean] = $sess.cmd17Wrapper$Helper$$Lambda$3658/1808578780@7d8437bf

#### Primera prueba

Dos transferencias lanzadas en paralelo. El mismo importe (`500`) desde la cuenta 1 a la 2 y desde la 2 a la 1

In [19]:
TestUtil.result{
    
    transfersOneToTwo( 500 ).zipWith( transfersTwoToOne( 300 ) ) {
        (a, b) =>  a && b
    }
    
}

res18: Try[Boolean] = Success(true)

#### Se vuelen a compruebar los balances

> `Await` sólo por motivos de testing

In [ ]:
TestUtil.result {
    accountOne.getBalance.zipWith( accountTwo.getBalance ){
        ( b1, b2 ) => (b1, b2, b1 +b2) 
    }
}

#### Se vuelven a comprobar los eventos

In [21]:
queueCQRS.toList ; queueCQRS.clear

res20_0: List[AccountEvent] = List()

---
### Bonus track
Test de concurrencia, lanzando transferencias en paralelo desde las mismas cuentas con cantidades aleatorias.

El resultado debera conservar la suma de los balances y ninguna de las dos cuentas puede tener valores negativos, ni valores mayores que la suma de los dos balances inciales

In [22]:
object TestConcurrent {

    import scala.util.Random
    
    private def randomAmount = Random.nextInt( 998 ) + 1 
    
    private def randomOperation( amount : Amount ) =  {
        if ( Random.nextInt(2) == 0 ) {
            transfersOneToTwo( amount )
        } else {
            transfersTwoToOne( amount )
        }
    }

    private def randomTransfer( implicit ec : ExecutionContext ) = Future {
        randomOperation( randomAmount )
    }.flatten.map{
       _ => 1  
    } .recover{
        case _=> 0
    }
  
    def testConncurrent( cont: Int, 
                         from : List[Future[Int]] = List())( implicit ec : ExecutionContext ): Future[Int] ={

        if ( cont == 0 ) {
            Future.sequence( from ) .map {
                _.fold(0) { 
                        ( a,b ) => a + b 
                } 
            }
        } else  {
           testConncurrent( cont -1, from :+ randomTransfer )
        }
    }

}

defined object TestConcurrent

#### Se lanza el test
Se lanza `500` transferencias aleatorias y se comprueban los balances de las dos cuentas

In [23]:
TestUtil.result {
    for {
        t <- TestConcurrent.testConncurrent( 500 ) 
        ( b1, b2 ) <- accountOne.getBalance.zip( accountTwo.getBalance )
    } yield{  (t, (b1, b2, b1 +b2) ) }    
}

res22: Try[(Int, (Balance, Balance, Int))] = Success((29, (1126, 874, 2000)))

#### Se vuelven a comprobar los eventos

In [24]:
queueCQRS.toList ; queueCQRS.clear

res23_0: List[AccountEvent] = List(
  WithdrawalCreated(accountTwo,535),
  WithdrawalCreated(accountOne,595),
  WithdrawalCreated(accountOne,157),
  WithdrawalCreated(accountTwo,276),
  WithdrawalCreated(accountOne,13),
  WithdrawalCreated(accountTwo,207),
  WithdrawalCreated(accountTwo,21),
  WithdrawalCreated(accountTwo,87),
  WithdrawalCreated(accountTwo,69),
  WithdrawalCreated(accountOne,16),
  WithdrawalCreated(accountTwo,3),
...